# CIDS Custom Class

This notebook creates a centralized intrusion detection system (CIDS) using a custom class implemented in our GitHub module.

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# download SynCAN dataset repository, combine train files, and clean directory
!git clone https://github.com/etas/SynCAN.git &> /dev/null
%cd /content/SynCAN
!unzip \*.zip -d . &> /dev/null
!cat train_*.csv > train.csv
# !cat train_1.csv > train.csv
!rm *.zip &> /dev/null
%cd /content

# download project repository
!git clone https://github.com/brendon-felix/VehicleCybersecFL.git &> /dev/null
!rm ./VehicleCybersecFL/*.ipynb &> /dev/null
!cp -r ./VehicleCybersecFL/anomalydetection/* .
%run anomalydetection.py

In [ ]:
from psutil import virtual_memory
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(f'GPU: {gpu_info[599:619]}')
ram_gb = virtual_memory().total / 1e9
print(f'RAM: {ram_gb:.1f} GB')

In [ ]:
id = 2      # message ID must be between 1 and 10
signal_counts = [2, 3, 2, 1, 2, 2, 2, 1, 1, 4]
num_signals = signal_counts[id-1]
time_steps = 20
params = {
    'msg_id': 'id'+str(id),
    'train_split': 0.9,
    'val_split': 0.05,
    'time_steps': time_steps,
    'warm_up': 0,
    'seq_stride': time_steps,
    'batch_size': 128,
    'drop_out': True,
    'attention': True,
    'patience': 1,
    'input_dim': num_signals,
    'latent_dim': 64,
    'loss_function': 'mse',
    'learning_rate': 0.0001,
    'metric': 'mae',
    # 'model_dir': '/content/drive/MyDrive/Senior Design 22-23/Models/CIDS'+str(time_steps)+'/id'+str(id)+'/',
    'model_dir': '/content/drive/MyDrive/Senior Design 22-23/Models/test/',
}

# Prepare Data


In [ ]:
train_df = import_data('SynCAN/train.csv', params['msg_id'])
# df

29,669,725 total messages (id1,id2,...,id10)
2,070,144 messages used (id2)
0 anomalous messages out of 2,070,144



In [ ]:
visualize_data(train_df, end_time=5e5)

# Create and Train Model


In [ ]:
cm = CentralizedModel(train_df, params, verbose=True)
# cm.model.summary()

Train:
93,156 subsequences of length 20
728 batches (batch size 128)

Val:
5,175 subsequences of length 20
41 batches (batch size 128)

Test:
5,175 subsequences of length 20
41 batches (batch size 128)

Saving model to /content/drive/MyDrive/Senior Design 22-23/Models/test/
Initializing centralized model...



In [ ]:
train_df.iloc[:,1:].to_numpy().min(), train_df.iloc[:,1:].to_numpy().max(), train_df.iloc[:,1:].to_numpy().mean()

(0.0, 1.0, 0.32578117915173127)

In [ ]:
cm.train_model(epochs=15, plot_loss=True, evaluate=True)

Epoch 1/15
728/728 [==============================] - 30s 27ms/step - loss: 0.0112 - mae: 0.0554 - val_loss: 0.0012 - val_mae: 0.0218
Epoch 2/15
728/728 [==============================] - 17s 24ms/step - loss: 0.0014 - mae: 0.0277 - val_loss: 6.1857e-04 - val_mae: 0.0150
Epoch 3/15
728/728 [==============================] - 18s 25ms/step - loss: 9.6774e-04 - mae: 0.0234 - val_loss: 3.9673e-04 - val_mae: 0.0124
Epoch 4/15
728/728 [==============================] - 19s 26ms/step - loss: 8.0095e-04 - mae: 0.0211 - val_loss: 3.2655e-04 - val_mae: 0.0108
Epoch 5/15
728/728 [==============================] - 18s 24ms/step - loss: 7.1594e-04 - mae: 0.0197 - val_loss: 2.8891e-04 - val_mae: 0.0103
Epoch 6/15
728/728 [==============================] - 18s 24ms/step - loss: 6.4915e-04 - mae: 0.0186 - val_loss: 2.6480e-04 - val_mae: 0.0098
Epoch 7/15
728/728 [==============================] - 19s 26ms/step - loss: 5.7672e-04 - mae: 0.0176 - val_loss: 2.2087e-04 - val_mae: 0.0091
Epoch 8/15
301/728

KeyboardInterrupt: ignored